<a href="https://colab.research.google.com/github/3hoodaas/Mobadron_AI/blob/main/Mobadron_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# الخطوة 1: استيراد المكتبات
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import joblib

# الخطوة 2: تحميل البيانات

file_path = '/content/Fake_Initiatives_Dataset_500.xlsx'  # استبدل هذا بالمسار الخاص بك
data = pd.read_excel(file_path)

# عرض أول 5 صفوف من البيانات
print("عرض أول 5 صفوف من البيانات:")
print(data.head())

# الخطوة 3: تنظيف البيانات
# التأكد من عدم وجود قيم مفقودة
print("\nالتأكد من القيم المفقودة:")
print(data.isnull().sum())

# إذا كانت هناك قيم مفقودة، يمكن استبدالها أو حذفها
# حذف الصفوف التي تحتوي على قيم مفقودة
data.dropna(inplace=True)
print("\nبعد تنظيف القيم المفقودة:")
print(data.isnull().sum())

# الخطوة 4: معالجة النصوص باستخدام LabelEncoder
label_encoder = LabelEncoder()

# الأعمدة النصية التي تحتاج إلى تحويل
text_columns = ['Target_Group', 'Expected_Impact']  # مثال
for col in text_columns:
    data[col] = label_encoder.fit_transform(data[col])

# الخطوة 5: إضافة عمود للأولوية باستخدام النقاط الموزونة بدون عمود Time
def calculate_priority_weighted_no_time(row):
    # تحديد نقاط الميزانية
    if row['Budget'] > 3000000:
        budget_score = 10
    elif row['Budget'] > 2000000:
        budget_score = 7
    elif row['Budget'] > 1000000:
        budget_score = 5
    else:
        budget_score = 3

    # تحديد نقاط عدد المستفيدين
    if row['Beneficiaries'] > 20000:
        beneficiaries_score = 10
    elif row['Beneficiaries'] > 10000:
        beneficiaries_score = 7
    elif row['Beneficiaries'] > 5000:
        beneficiaries_score = 5
    else:
        beneficiaries_score = 3

    # تحديد نقاط التأثير المتوقع
    if row['Expected_Impact'] > 50000:
        impact_score = 10
    elif row['Expected_Impact'] > 20000:
        impact_score = 7
    elif row['Expected_Impact'] > 10000:
        impact_score = 5
    else:
        impact_score = 3

    # حساب النقاط النهائية باستخدام الأوزان
    final_score = (0.333 * budget_score) + (0.333 * beneficiaries_score) + \
                  (0.333 * impact_score)

    # تحديد الأولوية بناءً على النقاط النهائية
    if final_score > 8:
        return 1  # أولوية عالية
    elif final_score >= 5:
        return 2  # أولوية متوسطة
    else:
        return 3  # أولوية منخفضة

# تطبيق المعادلة الجديدة على البيانات
data['Priority'] = data.apply(calculate_priority_weighted_no_time, axis=1)


# الخطوة 6: إعداد الميزات والهدف
features = ['Target_Group', 'Budget', 'Beneficiaries']  # الميزات الأساسية
X = data[features]
y = data['Priority']

# تقسيم البيانات إلى بيانات تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تجهيز الميزات باستخدام StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# الخطوة 7: إنشاء النموذج
model = RandomForestClassifier(random_state=42)

# تدريب النموذج
model.fit(X_train_scaled, y_train)

# التنبؤ بالاختبار
y_pred = model.predict(X_test_scaled)

# الخطوة 8: تقييم الأداء
print("\nتقرير الأداء للنموذج:")
print(classification_report(y_test, y_pred))

# الخطوة 9: حفظ النموذج
model_path = '/content/recommendation_model.pkl'
joblib.dump(model, model_path)
print(f"\nتم حفظ النموذج في: {model_path}")

# الخطوة 10: حفظ البيانات المعدلة
output_file_path = '/content/Modified_Initiatives_with_Priority.xlsx'
data.to_excel(output_file_path, index=False)
print(f"\nتم حفظ البيانات المعدلة في: {output_file_path}")


عرض أول 5 صفوف من البيانات:
  Initiative_Name Target_Group  Expected_Impact   Budget  Beneficiaries
0    Initiative 1     Startups            78434  3089471          14013
1    Initiative 2        Women            67784  2055760          20398
2    Initiative 3     Startups            74902  4267869           7937
3    Initiative 4        Women            29707  2056631          45692
4    Initiative 5        Women            18907   354684          13875

التأكد من القيم المفقودة:
Initiative_Name    0
Target_Group       0
Expected_Impact    0
Budget             0
Beneficiaries      0
dtype: int64

بعد تنظيف القيم المفقودة:
Initiative_Name    0
Target_Group       0
Expected_Impact    0
Budget             0
Beneficiaries      0
dtype: int64

تقرير الأداء للنموذج:
              precision    recall  f1-score   support

           2       0.99      0.98      0.98        86
           3       0.87      0.93      0.90        14

    accuracy                           0.97       100
   macro 